In [1]:
%load_ext autotime

In [3]:
import numpy as np 
import matplotlib as mpl 
import matplotlib.pyplot as plt 
import pickle
import random

from tqdm import tqdm
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

import os, shutil, glob, os.path

data_trainX = np.loadtxt('train_triplets.txt')
data_testX = np.loadtxt('test_triplets.txt')

time: 1.06 s


In [4]:
# load pretrained model: imagenet
image.LOAD_TRUNCATED_IMAGES = True 
model = VGG16(weights='imagenet', include_top=False)

imdir = 'food/'
filelist = glob.glob(os.path.join(imdir, '*.jpg'))
filelist.sort()

featurelist = []
for i, imagepath in enumerate(filelist):
    try:
        print("    Status: %s / %s" %(i, len(filelist)), end="\r")
        img = image.load_img(imagepath, target_size=(224, 224))
        img_data = image.img_to_array(img)
        img_data = np.expand_dims(img_data, axis=0)
        img_data = preprocess_input(img_data)
        features = np.array(model.predict(img_data))
        featurelist.append(features.flatten())
    except:
        continue

# optional: save model in pickle file
# with open('featurelist_vgg16', 'wb') as fp:
#    pickle.dump(featurelist, fp)

Instructions for updating:
Colocations handled automatically by placer.
time: 1h 36min 27s 10000


In [8]:
def similarity_numbers (itemlist, triplets, triplets_test, ncat):
    '''
    This function computes a grid in which the elements are pairs of classes.
    The elements of the grid are computed using triplets, which is an array 
    containing triplets. +1 is summed in the entry of the grid corresponding
    to similar categories of each triplet element and -1 is summed when pairs
    are less similar. The grid is used to make predictions on triples_test which
    has the same structure as triplets.
    '''
    np.random.seed(4)
    pairs = np.zeros((ncat, ncat))
    
    for i in range(len(triplets)):
        first = itemlist[int(triplets[i][0])]
        second = itemlist[int(triplets[i][1])]
        third = itemlist[int(triplets[i][2])]
        pairs[first,second] += 1
        pairs[second,first] += 1
        pairs[first,third] -= 1
        pairs[third,first] -= 1
    
    predictions = []
    for i in range(len(triplets_test)):
        first_test = itemlist[int(triplets_test[i][0])]
        second_test = itemlist[int(triplets_test[i][1])]
        third_test = itemlist[int(triplets_test[i][2])]
    
        comparison_1 = pairs[first_test,second_test]
        comparison_2 = pairs[first_test,third_test]
        
        if  comparison_1 > comparison_2:
            predictions.append(1)
        elif comparison_1 == comparison_2:
            predictions.append(np.random.randint(2))
        else:
            predictions.append(0)
        
    return np.array(predictions)

time: 5.13 ms


In [ ]:
# optional: load pickle file 
# with open ('featurelist_vgg16', 'rb') as fp:
#    featurelist = pickle.load(fp)

"""We figured that most of the predictions done by setting a fixed value for the PCA components 
and the number of clusters in the Kmeans performed comparably well in cv. However this predictions are quite
different from one another. For this reason we exploited this diversity through the usage on an
averaging technique. To have many predictions we ran two for loops, over PCA dimension and number of
clusters in Kmeans. To ensure diversity of the predictions we measured distance between predictions 
and set the threshold = 20000, using cv. We then transformed the average values > 0.5 to 1 and < 0.5 to 0. 
This raised the cv score from 0.60 to 0.64.
"""

final = np.zeros(len(data_testX))

threshold = 20000
counter = 0
M = []
first = True
for components in range(10,110,10):
    print('PCA: ',components)
    pca = PCA(n_components=components)
    images_pca = pca.fit_transform(featurelist)
    cluster_range = np.arange(10,110)
    random.shuffle(cluster_range)
    
    for clusters in cluster_range:
        print('cluster: ',clusters)
        kmeans = KMeans(n_clusters=clusters, random_state=0).fit(np.array(images_pca))
        pred = similarity_numbers(kmeans.labels_, data_trainX, data_testX, clusters)
        if first: 
            M.append(pred)
            counter += 1
        first = False
        check = True
        for i in range(len(M)):
            a = (np.linalg.norm(M[i]-pred))**2
            if a < threshold: 
                check = False
                
        if check: 
            M.append(pred)
            counter += 1

M = np.array(M)
for i in range(len(M)):
    final += M[i]/len(M)

predictions = np.zeros(len(final))
for i in range(len(final)):
    if final[i] > 0.5: predictions[i] = 1
        
np.savetxt("predictions.txt", predictions, fmt="%i")